In [ ]:
#!pip3.10 install requests
#!pip3.10 install bs4
#!pip3.10 install pandas
#!pip3.10 install numpy

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
def getForecast(ticker):
    html_data = requests.get("https://money.cnn.com/quote/forecast/forecast.html?symb=" + ticker, headers={"User-Agent":"Mozilla/5.0"})
    html = BeautifulSoup(html_data.text, "html.parser")
    
    try:
        subdiv = html.find('div', attrs = {"class":"wsod_twoCol"})
        forecast_text = subdiv.find('p').text
        median_target = float(forecast_text.split("median target of")[1].split(',')[0])
        high_estimate = float(forecast_text.split("high estimate of")[1].split('and')[0])
        low_estimate = float(forecast_text.split("low estimate of")[1].split('.')[0])
        last_price = float(forecast_text.split("last price of")[1].split('.')[0])
        recc = html.find('strong', attrs = {"wsod_rating"})
        
        return f"{ticker}, {median_target}, {high_estimate}, {low_estimate}, {last_price}, {((median_target-last_price)/last_price) * 100}, {((low_estimate-last_price)/last_price) * 100}, {recc.get_text()}"
        
    except:
        return "Not found"

In [ ]:
df = pd.read_csv("Stocks in the Russell 1000 Index.csv")
df["Symbol"] = df["Symbol"].str.lower()


In [ ]:
# Create empty lists to store forecast data
median_targets = []
high_estimates = []
low_estimates = []
last_prices = []
percentage_changes_low = []  # Percentage change between low estimate and last price
percentage_changes_median = []  # Percentage change between median target and last price
recommendations = []

for symbol in df["Symbol"]:
    data = getForecast(symbol)  # Call the getForecast function to retrieve data
    
    if data != "Not found":
        data_split = data.split(", ")
        median_targets.append(float(data_split[1]))
        high_estimates.append(float(data_split[2]))
        low_estimates.append(float(data_split[3]))
        last_prices.append(float(data_split[4]))
        percentage_changes_median.append(f"{float(data_split[5])}%")
        percentage_changes_low.append(f"{float(data_split[6])}%")
        recommendations.append(data_split[7])

# Add new columns to the DataFrame
df["Median Target"] = median_targets
df["High Estimate"] = high_estimates
df["Low Estimate"] = low_estimates
df["Last Price"] = last_prices
df["Percentage Change (Median)"] = percentage_changes_median
df["Percentage Change (Low)"] = percentage_changes_low
df["Recommendation"] = recommendations